In [1]:
from preparing_data import *
import matplotlib.pyplot as plt
import os
import tensorflow as tf

%matplotlib inline

ZIP_PATH = os.path.join(os.getcwd(), 'data/numbers/archive.zip')
UNZIP_PATH = os.path.join(os.getcwd(), 'data/numbers/')

In [2]:
unzip_data(ZIP_PATH, UNZIP_PATH, True)

In [3]:
TRAINING_PATH = os.path.join(UNZIP_PATH, 'Train_Nums/')
TEST_PATH = os.path.join(UNZIP_PATH, 'Test_Nums/')

os.rename(TRAINING_PATH, os.path.join(UNZIP_PATH, 'Training/'))
os.rename(TEST_PATH, os.path.join(UNZIP_PATH, 'Test/'))

TEST_PATH = os.path.join(UNZIP_PATH, 'Test/')
TRAINING_PATH = os.path.join(UNZIP_PATH, 'Training/')
VALIDATION_PATH = os.path.join(UNZIP_PATH, 'Validation/')

assert os.path.isdir(TRAINING_PATH)

assert os.path.isdir(TEST_PATH)

In [4]:
split_test_val(TEST_PATH, VALIDATION_PATH, 0.7)

In [5]:
tr = data_counter(TRAINING_PATH, 'Training')
te = data_counter(TEST_PATH, 'Test')
vl = data_counter(VALIDATION_PATH, 'Validation')

Number of files in Training is 9900
Number of files in Test is 330
Number of files in Validation is 770


In [27]:

training_data = tf.keras.utils.image_dataset_from_directory(directory=TRAINING_PATH,
                                                labels='inferred',
                                                label_mode='int',
                                                color_mode='rgb',
                                                shuffle=True,
                                                batch_size=16,
                                                image_size=(100, 100))

validation_data = tf.keras.utils.image_dataset_from_directory(directory=VALIDATION_PATH,
                                                labels='inferred',
                                                label_mode='int',
                                                color_mode='rgb',
                                                shuffle=True,
                                                batch_size=16,
                                                image_size=(100, 100))

test_data = tf.keras.utils.image_dataset_from_directory(directory=TEST_PATH,
                                                labels='inferred',
                                                label_mode='int',
                                                color_mode='rgb',
                                                shuffle=True,
                                                batch_size=1,
                                                image_size=(100, 100))
                                        
assert training_data.class_names == validation_data.class_names == test_data.class_names

Found 9900 files belonging to 11 classes.
Found 770 files belonging to 11 classes.
Found 330 files belonging to 11 classes.


In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range=(0.1, 0.5),
                                   channel_shift_range=0.5,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   vertical_flip=True,
                                   horizontal_flip=True,
                                   rotation_range=45)

training_data = train_datagen.flow_from_directory(directory=TRAINING_PATH,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  target_size=(256, 256),
                                                  batch_size=16)

val_datagen = ImageDataGenerator(rescale=1./255)
validation_data = val_datagen.flow_from_directory(directory=VALIDATION_PATH,
                                                  class_mode='categorical',
                                                  target_size=(256, 256),
                                                  shuffle=True,
                                                  batch_size=16)

Found 9900 images belonging to 11 classes.
Found 770 images belonging to 11 classes.


In [9]:
names = training_data.class_names
names

AttributeError: 'DirectoryIterator' object has no attribute 'class_names'

In [10]:
plt.figure(figsize=(10, 10))
for images, labels in training_data.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(names[labels[i]])
    plt.axis("off")


AttributeError: 'DirectoryIterator' object has no attribute 'take'

<Figure size 720x720 with 0 Axes>

## Data Augumentation to prevent overfitting

In [15]:
data_augumentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomZoom(0.3),
    tf.keras.layers.RandomContrast(factor=(0.1, 0.9)),
    tf.keras.layers.RandomRotation(factor=(-0.2, 0.4))
])

# plt.figure(figsize=(10, 10))

# for images, _ in training_data.take(1):
#     for i in range(9):
#         augumented = data_augumentation(images)
#         ax = plt.subplot(3,3, i+1)
#         plt.imshow(augumented[0].numpy().astype('float32'))
#         plt.axis('off')

In [13]:
from keras.applications import ResNet152V2
from keras import Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import TensorBoard


class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.99:
            print(f'Training accuracy greater than or equal to 99% so stop training on epoch {epoch+1}.')
            self.model.stop_training = True
        elif logs.get('val_accuracy') >= 0.97:
            print(f'Validation accuracy greater than or equal to 97% so stop training on epoch {epoch+1}.')
            self.model.stop_trainig = True

callback = MyCallback()
!mkdir ./logs/resnet_numbers
tf_board_callback = TensorBoard(log_dir='./logs/resnet_numbers/', write_graph=True, update_freq='epoch')

input = Input(shape=(256, 256, 3))
x = ResNet152V2(include_top=False, weights=None, input_shape=(256, 256, 3))(input)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(11, activation='softmax')(x)

resnet_model = Model(input, x)

resnet_model.compile(optimizer='adam',
              metrics=['accuracy'],
              loss='categorical_crossentropy')
resnet_model.fit(training_data,
          epochs=20,
          validation_data=validation_data,
          callbacks=[callback, tf_board_callback])

/bin/bash: /home/barti/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: cannot create directory ‘./logs/resnet_numbers’: File exists
Epoch 1/20
270/619 [============>.................] - ETA: 1:50 - loss: 8.0642 - accuracy: 0.0875

2022-07-05 12:55:59.796669: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.10GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-07-05 12:55:59.848575: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-07-05 12:55:59.945180: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


619/619 [==============================] - 208s 321ms/step - loss: 4.8708 - accuracy: 0.0848 - val_loss: 2.4070 - val_accuracy: 0.0948
Epoch 2/20
125/619 [=====>........................] - ETA: 2:36 - loss: 2.3983 - accuracy: 0.0797

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

print(tf.device('CPU'))

: 